In [309]:
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import numpy as np

# Read data

In [310]:
norm = 'True' # or False

In [311]:
wcct = pd.read_csv('WCCT_data.csv')
wcct.head()

,VOLUNTEER,DAY,B.cells.plasma.STAT5+,Basophils,Bcells,Bcells.CSM,Bcells.NCSM,Bcells.plasma,CD66+,cMCs,...,Tcells.CD8+.CD161+,Tcells.CD8+.CD38+,Tcells.CD8+.CD38+Ki67+,Tcells.CD8+.Effector.CD38+,Tcells.CD8+.Effector.CD38+Ki67+,Tcells.CD8+.Memory,Tcells.CD8+.Memory.CD38+,Tcells.CD8+.Memory.CD38+Ki67+,Tcells.CD8+CD45RA+CD27-,label
0,101,Baseline 1,0.003108,1.712350,5.394990,1.103238,0.643297,0.046616,58.753327,25.943191,...,0.829759,0.180247,0.105662,0.015539,0.006215,4.583877,0.087016,0.065262,1.945429,1
1,101,Baseline 2,0.000000,1.545919,6.057033,1.218394,0.768593,0.026202,54.366529,24.429014,...,0.790428,0.231451,0.152845,0.017468,0.008734,4.908511,0.061138,0.056771,1.248963,1
2,101,1,0.000000,1.537663,5.168144,1.034428,0.571132,0.027958,66.719099,29.427271,...,0.635035,0.195703,0.131800,0.000000,0.000000,3.993929,0.095854,0.063903,1.282051,1
3,101,2,0.003162,1.479935,4.920469,0.961326,0.645100,0.044272,66.973242,26.581918,...,0.562881,0.252980,0.123328,0.006325,0.003162,4.003415,0.091705,0.050596,1.862568,1
4,101,3,0.000000,1.613706,6.140787,1.069585,0.609415,0.043530,64.295463,27.734594,...,0.513028,0.195883,0.071513,0.006219,0.003109,3.867919,0.090169,0.049748,1.402276,1


In [312]:
vxa = pd.read_csv('VXA_data.csv')
vxa.head()

,VOLUNTEER,DAY,B Cells Plasma STAT5+,Basophils,B Cells,B Cells CSM,B Cells NCSM,B Cells Plasma,CD66+,cMCs,...,CD8 T Cells CD161+,CD8 T Cells CD38+,CD8 T Cells CD38+Ki67+,CD8 T Cells CD45RA+CD27- CD38+,CD8 T Cells CD45RA+CD27- CD38+Ki67+,CD8 T Cells Memory,CD8 T Cells Memory CD38+,CD8 T Cells Memory CD38+Ki67+,CD8 T Cells CD45RA+CD27-,label
0,107,Baseline 1,0.004185,2.657627,8.008454,1.017013,1.332999,0.012556,145.606546,13.706657,...,4.436353,0.039760,0.033482,0.002093,0.004185,22.799088,0.108816,0.140205,1.697114,1
1,107,Baseline 2,0.030483,1.879795,6.987417,0.990703,1.251503,0.049112,160.244881,13.653068,...,3.381937,0.064353,0.123626,0.001694,0.001694,21.634575,0.193060,0.204915,2.003421,1
2,107,1,0.014398,1.906895,6.810110,1.047832,1.035034,0.025596,165.547912,15.797472,...,3.781795,0.035194,0.075188,0.001600,0.003199,19.644857,0.135978,0.148776,2.583587,1
3,107,2,0.010650,1.830298,5.430036,0.769851,0.874831,0.022822,196.138573,20.163707,...,3.167648,0.022822,0.065422,0.000000,0.003043,18.321238,0.135409,0.144537,2.697522,1
4,107,3,0.002682,2.012604,4.848485,0.662376,0.691874,0.010727,177.886833,23.716814,...,3.259587,0.049611,0.073746,0.000000,0.008045,17.035398,0.164924,0.151515,2.240547,1


# Preprocess data

In [313]:
def reverse_days(x):
    if x == 'Baseline 1':
        return -1
    if x == 'Baseline 2':
        return 1
    else:
        return int(x) +1

In [314]:
wcct['DAY'] = wcct['DAY'].apply(lambda x: reverse_days(x))
vxa['DAY'] = vxa['DAY'].apply(lambda x: reverse_days(x))

In [315]:
wcct.head()

,VOLUNTEER,DAY,B.cells.plasma.STAT5+,Basophils,Bcells,Bcells.CSM,Bcells.NCSM,Bcells.plasma,CD66+,cMCs,...,Tcells.CD8+.CD161+,Tcells.CD8+.CD38+,Tcells.CD8+.CD38+Ki67+,Tcells.CD8+.Effector.CD38+,Tcells.CD8+.Effector.CD38+Ki67+,Tcells.CD8+.Memory,Tcells.CD8+.Memory.CD38+,Tcells.CD8+.Memory.CD38+Ki67+,Tcells.CD8+CD45RA+CD27-,label
0,101,-1,0.003108,1.712350,5.394990,1.103238,0.643297,0.046616,58.753327,25.943191,...,0.829759,0.180247,0.105662,0.015539,0.006215,4.583877,0.087016,0.065262,1.945429,1
1,101,1,0.000000,1.545919,6.057033,1.218394,0.768593,0.026202,54.366529,24.429014,...,0.790428,0.231451,0.152845,0.017468,0.008734,4.908511,0.061138,0.056771,1.248963,1
2,101,2,0.000000,1.537663,5.168144,1.034428,0.571132,0.027958,66.719099,29.427271,...,0.635035,0.195703,0.131800,0.000000,0.000000,3.993929,0.095854,0.063903,1.282051,1
3,101,3,0.003162,1.479935,4.920469,0.961326,0.645100,0.044272,66.973242,26.581918,...,0.562881,0.252980,0.123328,0.006325,0.003162,4.003415,0.091705,0.050596,1.862568,1
4,101,4,0.000000,1.613706,6.140787,1.069585,0.609415,0.043530,64.295463,27.734594,...,0.513028,0.195883,0.071513,0.006219,0.003109,3.867919,0.090169,0.049748,1.402276,1


In [316]:
vxa.head()

,VOLUNTEER,DAY,B Cells Plasma STAT5+,Basophils,B Cells,B Cells CSM,B Cells NCSM,B Cells Plasma,CD66+,cMCs,...,CD8 T Cells CD161+,CD8 T Cells CD38+,CD8 T Cells CD38+Ki67+,CD8 T Cells CD45RA+CD27- CD38+,CD8 T Cells CD45RA+CD27- CD38+Ki67+,CD8 T Cells Memory,CD8 T Cells Memory CD38+,CD8 T Cells Memory CD38+Ki67+,CD8 T Cells CD45RA+CD27-,label
0,107,-1,0.004185,2.657627,8.008454,1.017013,1.332999,0.012556,145.606546,13.706657,...,4.436353,0.039760,0.033482,0.002093,0.004185,22.799088,0.108816,0.140205,1.697114,1
1,107,1,0.030483,1.879795,6.987417,0.990703,1.251503,0.049112,160.244881,13.653068,...,3.381937,0.064353,0.123626,0.001694,0.001694,21.634575,0.193060,0.204915,2.003421,1
2,107,2,0.014398,1.906895,6.810110,1.047832,1.035034,0.025596,165.547912,15.797472,...,3.781795,0.035194,0.075188,0.001600,0.003199,19.644857,0.135978,0.148776,2.583587,1
3,107,3,0.010650,1.830298,5.430036,0.769851,0.874831,0.022822,196.138573,20.163707,...,3.167648,0.022822,0.065422,0.000000,0.003043,18.321238,0.135409,0.144537,2.697522,1
4,107,4,0.002682,2.012604,4.848485,0.662376,0.691874,0.010727,177.886833,23.716814,...,3.259587,0.049611,0.073746,0.000000,0.008045,17.035398,0.164924,0.151515,2.240547,1


In [317]:
if norm == 'True':
    wcct.loc[:,  (wcct.columns != 'VOLUNTEER') &  (wcct.columns != 'DAY') & (wcct.columns != 'label')] = \
        wcct.loc[:,  (wcct.columns != 'VOLUNTEER') &  (wcct.columns != 'DAY') & (wcct.columns != 'label')].subtract(
        (wcct.groupby('VOLUNTEER').transform(lambda x:x.iloc[0])))#.round(2)

In [318]:
if norm == 'True':
    vxa.loc[:,  (vxa.columns != 'VOLUNTEER') &  (vxa.columns != 'DAY') & (vxa.columns != 'label')] = \
        vxa.loc[:,  (vxa.columns != 'VOLUNTEER') &  (vxa.columns != 'DAY') & (vxa.columns != 'label')].subtract(
        (vxa.groupby('VOLUNTEER').transform(lambda x:x.iloc[0])))#.round(2)

# Match cell populations

In [319]:
# use common populations
mcl = pd.read_excel('matched_populations.xlsx')
wcct_cols = mcl['WCCT'].values.tolist()
wcct_cols.extend(['VOLUNTEER', 'DAY', 'label'])
vxa_cols = mcl['VXA'].values.tolist()
vxa_cols.extend(['VOLUNTEER', 'DAY', 'label'])
mcl.head()

,index,VXA,WCCT
0,1,B Cells Plasma STAT5+,B.cells.plasma.STAT5+
1,2,Basophils,Basophils
2,3,B Cells,Bcells
3,4,B Cells CSM,Bcells.CSM
4,5,B Cells NCSM,Bcells.NCSM


In [320]:
wcct = wcct[wcct_cols]
vxa = vxa[vxa_cols]

In [321]:
wcct.head()

,B.cells.plasma.STAT5+,Basophils,Bcells,Bcells.CSM,Bcells.NCSM,Bcells.plasma,CD66+,cMCs,intMCs,mDC,...,Tcells.CD8+.CD38+Ki67+,Tcells.CD8+.Effector.CD38+,Tcells.CD8+.Effector.CD38+Ki67+,Tcells.CD8+.Memory,Tcells.CD8+.Memory.CD38+,Tcells.CD8+.Memory.CD38+Ki67+,Tcells.CD8+CD45RA+CD27-,VOLUNTEER,DAY,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,101,-1,1
1,-0.003108,-0.166431,0.662043,0.115156,0.125296,-0.020414,-4.386798,-1.514177,-0.642958,0.123234,...,0.047183,0.001929,0.002519,0.324634,-0.025878,-0.008491,-0.696466,101,1,1
2,-0.003108,-0.174687,-0.226846,-0.068811,-0.072165,-0.018658,7.965772,3.484080,-0.741623,0.519418,...,0.026137,-0.015539,-0.006215,-0.589948,0.008838,-0.001359,-0.663377,101,2,1
3,0.000055,-0.232415,-0.474521,-0.141913,0.001803,-0.002344,8.219915,0.638727,-0.879535,1.077903,...,0.017666,-0.009214,-0.003053,-0.580462,0.004689,-0.014666,-0.082860,101,3,1
4,-0.003108,-0.098644,0.745797,-0.033653,-0.033882,-0.003086,5.542136,1.791403,-0.838885,0.572898,...,-0.034149,-0.009320,-0.003106,-0.715959,0.003153,-0.015514,-0.543153,101,4,1


In [322]:
vxa.head()

,B Cells Plasma STAT5+,Basophils,B Cells,B Cells CSM,B Cells NCSM,B Cells Plasma,CD66+,cMCs,intMCs,mDCs,...,CD8 T Cells CD38+Ki67+,CD8 T Cells CD45RA+CD27- CD38+,CD8 T Cells CD45RA+CD27- CD38+Ki67+,CD8 T Cells Memory,CD8 T Cells Memory CD38+,CD8 T Cells Memory CD38+Ki67+,CD8 T Cells CD45RA+CD27-,VOLUNTEER,DAY,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,107,-1,1
1,0.026298,-0.777832,-1.021037,-0.026310,-0.081496,0.036556,14.638336,-0.053589,-0.223652,0.053221,...,0.090144,-0.000399,-0.002492,-1.164513,0.084244,0.064709,0.306307,107,1,1
2,0.010212,-0.750732,-1.198344,0.030819,-0.297964,0.013040,19.941367,2.090816,0.109187,0.165704,...,0.041706,-0.000493,-0.000986,-3.154231,0.027162,0.008571,0.886472,107,2,1
3,0.006465,-0.827328,-2.578418,-0.247162,-0.458168,0.010266,50.532027,6.457051,0.978653,0.155394,...,0.031940,-0.002093,-0.001142,-4.477850,0.026592,0.004332,1.000407,107,3,1
4,-0.001504,-0.645023,-3.159969,-0.354637,-0.641124,-0.001829,32.280287,10.010158,3.759264,-0.067880,...,0.040264,-0.002093,0.003860,-5.763689,0.056107,0.011310,0.543433,107,4,1


In [323]:
# filter unwanted days out
tr_days = [2,3,4,5,6,7,8]
wcct = wcct[(wcct['DAY'].isin(tr_days)) ]
vxa = vxa[(vxa['DAY'].isin(tr_days))]

In [324]:
def evaluate_preds(true, pred):
    auc = roc_auc_score(true, pred)
    pr = average_precision_score(true, pred)
    bin_pred = [1 if p > 0.5 else 0 for p in pred]
    f_score = f1_score(true, bin_pred)
    return auc, pr, f_score

## Random Forest

# Classification

In [325]:
from sklearn.metrics import roc_auc_score
tr_days = [2,3,4,5,6,7,8]
tr_x = wcct[wcct['DAY'].isin(tr_days)].drop(['VOLUNTEER', 'DAY', 'label'], axis =1)
tr_y = wcct[wcct['DAY'].isin(tr_days)]['label']
for d in [2,3,4,6]:
    testx = vxa[vxa['DAY']==d].drop(['VOLUNTEER', 'DAY', 'label'], axis =1)
    testy =  vxa[vxa['DAY']==d]['label']
    aucs = []
    for i in range(10): # change 10 for bootstrapping
        RF = RandomForestClassifier(random_state = random.seed(1234))
        RF.fit(tr_x, tr_y)
        probs = RF.predict_proba(testx)  
        probs = probs[:, 1]  
        auc_ = roc_auc_score(testy, probs)
        aucs.append(auc_)
    #print('day %s'%d, np.mean(aucs))

# Filter shedders

In [326]:
wcct = wcct[(wcct['label']==1) & (~wcct['VOLUNTEER'].isin([101,109,304]))]
vxa = vxa[vxa['label']==1]

In [327]:
wcct.head()

,B.cells.plasma.STAT5+,Basophils,Bcells,Bcells.CSM,Bcells.NCSM,Bcells.plasma,CD66+,cMCs,intMCs,mDC,...,Tcells.CD8+.CD38+Ki67+,Tcells.CD8+.Effector.CD38+,Tcells.CD8+.Effector.CD38+Ki67+,Tcells.CD8+.Memory,Tcells.CD8+.Memory.CD38+,Tcells.CD8+.Memory.CD38+Ki67+,Tcells.CD8+CD45RA+CD27-,VOLUNTEER,DAY,label
24,-0.051331,-0.320943,-0.561717,-0.027045,0.021157,0.007392,-2.245394,4.438926,-0.117166,0.202109,...,0.025071,0.012010,0.008966,0.000374,0.067021,0.042542,-0.440921,103,2,1
25,-0.030780,-0.167358,-0.127951,-0.058401,0.060127,0.005042,-1.324993,3.512354,-0.280401,0.520787,...,0.012971,0.008112,0.005324,-0.080260,0.085435,0.006378,-0.213460,103,3,1
26,-0.040322,-0.364959,-0.414351,-0.135249,0.002995,-0.056090,0.124590,8.065160,0.290708,0.171458,...,0.109093,0.007139,0.004107,0.037023,0.140660,0.051878,-0.292690,103,4,1
27,-0.097732,-0.587658,-1.068621,-0.172138,-0.118536,-0.035305,-2.189082,7.526682,1.344945,0.127305,...,0.152269,0.029945,0.003421,0.171135,0.280960,0.093119,0.662680,103,5,1
28,-0.078188,-0.645096,-0.641137,-0.104277,-0.108517,0.038055,-4.108122,6.297360,1.533281,-0.019842,...,0.129443,0.040351,0.012268,0.107582,0.284179,0.048696,0.304579,103,6,1


In [328]:
vxa.head()

,B Cells Plasma STAT5+,Basophils,B Cells,B Cells CSM,B Cells NCSM,B Cells Plasma,CD66+,cMCs,intMCs,mDCs,...,CD8 T Cells CD38+Ki67+,CD8 T Cells CD45RA+CD27- CD38+,CD8 T Cells CD45RA+CD27- CD38+Ki67+,CD8 T Cells Memory,CD8 T Cells Memory CD38+,CD8 T Cells Memory CD38+Ki67+,CD8 T Cells CD45RA+CD27-,VOLUNTEER,DAY,label
2,0.010212,-0.750732,-1.198344,0.030819,-0.297964,0.013040,19.941367,2.090816,0.109187,0.165704,...,0.041706,-0.000493,-0.000986,-3.154231,0.027162,0.008571,0.886472,107,2,1
3,0.006465,-0.827328,-2.578418,-0.247162,-0.458168,0.010266,50.532027,6.457051,0.978653,0.155394,...,0.031940,-0.002093,-0.001142,-4.477850,0.026592,0.004332,1.000407,107,3,1
4,-0.001504,-0.645023,-3.159969,-0.354637,-0.641124,-0.001829,32.280287,10.010158,3.759264,-0.067880,...,0.040264,-0.002093,0.003860,-5.763689,0.056107,0.011310,0.543433,107,4,1
5,-0.000616,-1.094253,-2.906668,-0.482801,-0.603662,-0.000658,-41.516075,6.016362,6.384209,-0.011801,...,0.098584,0.006236,0.013661,-2.032630,0.126761,0.117977,0.319566,107,6,1
6,0.088270,-1.527874,-2.587447,-0.169135,-0.261876,0.111469,-75.566407,0.849335,0.792735,0.310580,...,1.301475,0.013692,0.034150,1.996835,1.086331,1.715655,0.772105,107,8,1


# Regression

In [329]:
from scipy import stats

regr = RandomForestRegressor(random_state = random.seed())
trus = []
preds = []
to_write = pd.DataFrame()
X_tr = wcct.drop(['VOLUNTEER','DAY', 'label'], axis =1)
Y_tr = wcct['DAY']
clf = regr.fit(X_tr, Y_tr)
vxa_ids = list(set(vxa['VOLUNTEER'].values.tolist()))
vxa_days = list(set(vxa['DAY'].values.tolist()))
for n,id_ in enumerate(vxa_ids):
    for d in vxa_days:
        test = vxa[(vxa['VOLUNTEER']==id_) & (vxa['DAY']==d)]#.sample(n=1)
        X_te = test.drop(['VOLUNTEER','DAY', 'label'], axis =1)
        Y_te = test['DAY']
        if len(X_te)==0:
            continue
        pr = regr.predict(X_te)
        trus.extend(Y_te.values)
        preds.extend(pr)
        dic_ = {}
        dic_['iteration'] = d
        dic_['ID'] = id_
        dic_['true'] = Y_te.values
        dic_['predicted'] = pr
        to_write = to_write.append(dic_, ignore_index=True)


In [330]:
correlation, p_value = stats.pearsonr(trus,preds)